This notebook uses iFeatureOmega, a feature generation software, to add to our feature space for a RandomForestClassifier that predicts protein pair functionality.

To do:

1) Write unit tests for iFeatureOmega
2) time trial for different descriptors - DONE
3) Figure out how to append meso and thermo descriptors
4) Make sure protein length is in training
5) try changing subject_align_len to subject_align_len/m_protein_len
6) log(ratio) for proteins approacing zero or infinity
    take distributions to assess for this.
    Update -- taking log and then droping infinites and NaN improves score to 0.86 <-- cannot repeat this

In [44]:
import duckdb

con = duckdb.connect(path_to_db)
cmd = """SELECT list_of_column_names FROM fafsa_final"""
df = con.execute(cmd).df()

NameError: name 'path_to_db' is not defined

In [ ]:
output_df.to_parquet(path)
cmd = f"""CREATE OR REPLACE TABLE table_name AS SELECT * FROM '{path}'"""
con.execute(cmd)

 protein_pair_cmd = """CREATE OR REPLACE TABLE fafsa_protein_pairs AS
                          SELECT 
                          meso_pid,
                          thermo_pid,
                          bit_score,
                          local_gap_compressed_percent_id,
                          scaled_local_query_percent_id,
                          scaled_local_symmetric_percent_id,
                          query_align_len,
                          query_align_cov,
                          subject_align_len,
                          subject_align_cov,
                          FROM protein_pairs USING SAMPLE 100
"""

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import scipy.stats
import sklearn.preprocessing
import sklearn.model_selection
import sklearn.ensemble
import sklearn.feature_selection
import unittest
import iFeatureOmegaCLI
import Bio.SeqIO
import Bio.SeqRecord
import io
from io import StringIO
import time

In [2]:
def get_fasta_from_dataframe(dataframe, output_file_a, output_file_b):
    #adjust this to write function with BioPython
    #separate functions for each of the input sequences
    #in training, seq_a = meso and seq_b = thermo
    
    
    #meso sequence to fasta
    with open(output_file_a, 'w') as f:
        for _, row in df.iterrows():
            f.write('>{}\n{}\n'.format((row['prot_pair_index']), row['m_protein_seq']))
    
    #thermo sequence to fasta
    with open(output_file_b, 'w') as f:
        for _, row in df.iterrows():
            f.write('>{}\n{}\n'.format((row['prot_pair_index']), (row['t_protein_seq'])))
   
    #return output files
    return [output_file_a, output_file_b]

In [3]:
def get_protein_descriptors(fasta_file, descriptors=[]):
    
    """
    Generates features from a protein sequence

    Parameters
    ----------
    Fasta file with amino acid sequences.

    Returns
    -------
    Vector of descriptors
    """
    
    #create iProtein object
    protein = iFeatureOmegaCLI.iProtein(fasta_file)
    
    #not sure why we need this yet. Right now it is stored in local directory.
    params = protein.import_parameters('protein_parameters.json')
    
    protein_descriptors = {}
    
    for descriptor in descriptors:
        protein.get_descriptor(descriptor)
        protein_descriptors.update({f'{descriptor}':protein.encodings})
        
    return protein_descriptors

In [4]:
def create_new_dataframe(dataframe, output_files, descriptors=[]):
    """
    Creates new dataframe with descriptors added.

    Parameters
    ----------
    Pandas dataframe, list of descriptors as strings, output file name.

    Returns
    -------
    Dataframe including vector(s) of descriptors
    """

    fasta_files = get_fasta_from_dataframe(dataframe, output_files[0], output_files[1])
    
    df = dataframe.reset_index()
    
    def compute_descriptor_ratio(fasta_files, descriptors=[]):
        """
        Generates dictionary of descriptors for each of the two input sequences.
        Computes the difference between each instance of a descriptor.
        
        Parameters
        ----------
        List of two fasta files (str) and list of descriptors (str).

        Returns
        -------
        Dictionary with difference between descriptors for each of the 
        input sequences.
        """
        desc_a = get_protein_descriptors(fasta_files[0], descriptors)
        desc_b = get_protein_descriptors(fasta_files[1], descriptors)

        feature_dict = {}

        for key in desc_a:
                        
            if 'GDPC type 2' in key:
                feature_dict[key] = np.log(desc_a[key] - desc_b[key])
                
            elif 'CKSAAGP type 2' in key:
                feature_dict[key] = np.log(desc_a[key] - desc_b[key])
                
            else:
                feature_dict[key] = (desc_a[key]) / desc_b[key]

        return feature_dict
    
    feature_dict = compute_descriptor_ratio(fasta_files, descriptors)

    for desc in descriptors:

        feature_dict[desc].index = feature_dict[desc].index.astype(int)
        features = feature_dict[desc].reset_index()

        df = pd.merge(
            df,
            features,
            how='outer',
            left_index=True,
            right_index=True)

    return df

In [5]:
#iFeature properties

protein = iFeatureOmegaCLI.iProtein('meso_50k.fasta')
protein.display_feature_types()


        ----- Available feature types ------        
        
        AAC                                                Amino acid composition
        EAAC                                               Enhanced amino acid composition
        CKSAAP type 1                                      Composition of k-spaced amino acid pairs type 1 - normalized
        CKSAAP type 2                                      Composition of k-spaced amino acid pairs type 2 - raw count
        DPC type 1                                         Dipeptide composition type 1 - normalized
        DPC type 2                                         Dipeptide composition type 2 - raw count
        TPC type 1                                         Tripeptide composition type 1 - normalized
        TPC type 2                                         Tripeptide composition type 1 - raw count
        CTDC                                               Composition
        CTDT                                      

In [6]:
cd /Users/loganroberts/Learn2Therm/ValidProt/FAFSA

/Users/loganroberts/Learn2Therm/ValidProt/FAFSA


In [7]:
#convert to pandas df
df = pd.read_csv('learn2therm_sample_50k.csv')
df.columns

Index(['Unnamed: 0', 'local_gap_compressed_percent_id',
       'scaled_local_query_percent_id', 'scaled_local_symmetric_percent_id',
       'query_align_len', 'query_align_cov', 'subject_align_len',
       'subject_align_cov', 'bit_score', 'thermo_index', 'meso_index',
       'prot_pair_index', 'meso_protein_int_index', 'thermo_protein_int_index',
       'taxa_pair_index', 'local_gap_compressed_percent_id_16s',
       'scaled_local_query_percent_id_16s',
       'scaled_local_symmetric_percent_id_16s', 'query_align_cov_16s',
       'subject_align_cov_16s', 'bit_score_16s', 'm_ogt', 't_ogt',
       'ogt_difference', 'm_protein_seq', 't_protein_seq', 'm_protein_desc',
       't_protein_desc', 'm_protein_len', 't_protein_len'],
      dtype='object')

In [42]:
cd /Users/loganroberts/Learn2Therm/ValidProt/notebooks

/Users/loganroberts/Learn2Therm/ValidProt/notebooks


In [74]:
target = pd.read_csv('protein_match_50k')

In [75]:
target.drop(columns=["Unnamed: 0"], inplace=True)

In [76]:
target['protein_match'].value_counts()

Yes    40296
No      8554
Name: protein_match, dtype: int64

In [77]:
(target['Jaccard_Score'] < 0.60).value_counts()

False    25951
True     22899
Name: Jaccard_Score, dtype: int64

In [78]:
# Apply the mask to the DataFrame
target = target[target['Jaccard_Score'] <= 0.45]

In [79]:
target['protein_match'].value_counts()

No     8554
Yes    7545
Name: protein_match, dtype: int64

In [12]:
from sklearn.utils import resample

# Assuming your data is in a pandas DataFrame called 'data'
# Separate the majority and minority classes
majority_class = target[target['protein_match'] == 'Yes']
minority_class = target[target['protein_match'] == 'No']

# Undersample the majority class to match the number of minority class samples
n_samples = len(minority_class)
undersampled_majority = resample(majority_class, n_samples=n_samples, replace=False)

# Combine the undersampled majority class with the minority class
balanced_data = pd.concat([undersampled_majority, minority_class])

In [13]:
balanced_data

,prot_pair_index,protein_match,Jaccard_Score
15103,40402339,Yes,0.666667
8701,47080120,Yes,1.000000
43869,176178293,Yes,0.600000
30414,140789469,Yes,0.600000
39720,94620951,Yes,0.500000
...,...,...,...
48833,70996712,No,0.285714
48839,14456723,No,0.250000
48845,78849058,No,0.250000
48847,161110219,No,0.250000


In [81]:
df = pd.merge(df, target, on=['prot_pair_index'])
df.shape

(16099, 32)

In [82]:
"""
this list comes from a combination of reading through the features and determining which might be useful
and timing some of the feature generations. those that took more than 30ish seconds were eliminated
Also removed those that have really high dimensionality (>4000)
"""

feature_list = ['AAC', 'GAAC', 'DistancePair',
               'CTDC', 'CTDT', 'CTDD', 'CTriad', 'GDPC type 1', 'GDPC type 2',
                'CKSAAGP type 1', 'CKSAAGP type 2', 'PseKRAAC type 2', 'PseKRAAC type 3A','PseKRAAC type 7',
                'PseKRAAC type 9', 'Geary','APAAC', 'QSOrder']

In [83]:
# df = create_new_dataframe(df, 'seq_50k.fasta', descriptors= ['EGAAC', 'CKSAAGP type 1', 'CKSAAGP type 2',
#                                                             'GDPC type 1', 'GDPC type 2', 'TPC type 1', 'TPC type 2',
#                                                             'GTPC type 1', 'GTPC type 2', 'DPC type 1', 'DPC type 2',
#                                                             'Moran', 'Geary', 'NMBroto', 'AC', 'CC', 'ACC',
#                                                             'PAAC', 'APAAC'])

In [84]:
df = create_new_dataframe(df, ['seq_50k_a.fasta', 'seq_50k_b.fasta'], descriptors=['GAAC'])

File imported successfully.
File imported successfully.


Split data into dev and test, and then split that into train and validation.

In [85]:
#drop columns that don't exihibit signficant pearson correlation with bit_score

df = df.drop(columns = ['meso_index', 'meso_protein_int_index', 'local_gap_compressed_percent_id_16s', 
                        'scaled_local_query_percent_id_16s', 'scaled_local_symmetric_percent_id_16s',
                       'bit_score_16s', 'm_ogt', 't_ogt', 'taxa_pair_index', 'thermo_protein_int_index'
                       , 'prot_pair_index', 'ogt_difference', 'Jaccard_Score',
                       'query_align_cov_16s', 'subject_align_cov_16s',
                       'Unnamed: 0', 'thermo_index','m_protein_seq', 't_protein_seq', 
                       'm_protein_desc', 't_protein_desc'])

In [86]:
df=df.drop(columns=['index_y', 'index_x'])
df

,local_gap_compressed_percent_id,scaled_local_query_percent_id,scaled_local_symmetric_percent_id,query_align_len,query_align_cov,subject_align_len,subject_align_cov,bit_score,m_protein_len,t_protein_len,protein_match,GAAC_alphatic,GAAC_aromatic,GAAC_postivecharge,GAAC_negativecharge,GAAC_uncharge
0,0.275281,0.182156,0.152174,185,0.687732,197,0.525333,48,375,269,No,1.145322,0.868351,1.021657,1.099911,0.727882
1,0.354067,0.245847,0.225954,208,0.691030,216,0.610170,264,354,301,Yes,0.855840,1.700565,1.105367,1.118793,1.105367
2,0.270000,0.220408,0.222680,195,0.795918,210,0.875000,173,240,245,Yes,1.081477,0.850694,0.680556,0.922043,1.259752
3,0.268908,0.232727,0.247582,239,0.869091,236,0.975207,191,242,275,Yes,0.981818,1.217532,1.019814,1.409091,0.836490
4,0.253275,0.232000,0.230616,229,0.916000,227,0.897233,227,253,250,Yes,1.122453,0.790514,1.038816,1.018086,0.806983
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16094,0.284091,0.223214,0.245700,190,0.848214,175,0.956284,112,183,224,No,1.086768,0.979235,1.142441,0.559563,1.041739
16095,0.300000,0.300000,0.104803,80,1.000000,79,0.208995,104,378,80,Yes,0.898041,1.128748,1.096681,0.697168,1.787184
16096,0.344828,0.169972,0.192308,180,0.509915,200,0.738007,53,271,353,No,1.008708,2.388069,0.817901,1.302583,0.784208
16097,0.352174,0.239645,0.242152,230,0.680473,227,0.685801,279,331,338,No,1.045903,1.106244,0.799159,1.134609,0.943592


In [87]:
df.columns

Index(['local_gap_compressed_percent_id', 'scaled_local_query_percent_id',
       'scaled_local_symmetric_percent_id', 'query_align_len',
       'query_align_cov', 'subject_align_len', 'subject_align_cov',
       'bit_score', 'm_protein_len', 't_protein_len', 'protein_match',
       'GAAC_alphatic', 'GAAC_aromatic', 'GAAC_postivecharge',
       'GAAC_negativecharge', 'GAAC_uncharge'],
      dtype='object')

In [65]:
df=df.drop(columns=['local_gap_compressed_percent_id', 'scaled_local_query_percent_id',
       'scaled_local_symmetric_percent_id', 'query_align_len',
       'query_align_cov', 'subject_align_len', 'subject_align_cov',
       'bit_score', 'm_protein_len', 't_protein_len'])

In [88]:
df

,local_gap_compressed_percent_id,scaled_local_query_percent_id,scaled_local_symmetric_percent_id,query_align_len,query_align_cov,subject_align_len,subject_align_cov,bit_score,m_protein_len,t_protein_len,protein_match,GAAC_alphatic,GAAC_aromatic,GAAC_postivecharge,GAAC_negativecharge,GAAC_uncharge
0,0.275281,0.182156,0.152174,185,0.687732,197,0.525333,48,375,269,No,1.145322,0.868351,1.021657,1.099911,0.727882
1,0.354067,0.245847,0.225954,208,0.691030,216,0.610170,264,354,301,Yes,0.855840,1.700565,1.105367,1.118793,1.105367
2,0.270000,0.220408,0.222680,195,0.795918,210,0.875000,173,240,245,Yes,1.081477,0.850694,0.680556,0.922043,1.259752
3,0.268908,0.232727,0.247582,239,0.869091,236,0.975207,191,242,275,Yes,0.981818,1.217532,1.019814,1.409091,0.836490
4,0.253275,0.232000,0.230616,229,0.916000,227,0.897233,227,253,250,Yes,1.122453,0.790514,1.038816,1.018086,0.806983
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16094,0.284091,0.223214,0.245700,190,0.848214,175,0.956284,112,183,224,No,1.086768,0.979235,1.142441,0.559563,1.041739
16095,0.300000,0.300000,0.104803,80,1.000000,79,0.208995,104,378,80,Yes,0.898041,1.128748,1.096681,0.697168,1.787184
16096,0.344828,0.169972,0.192308,180,0.509915,200,0.738007,53,271,353,No,1.008708,2.388069,0.817901,1.302583,0.784208
16097,0.352174,0.239645,0.242152,230,0.680473,227,0.685801,279,331,338,No,1.045903,1.106244,0.799159,1.134609,0.943592


In [67]:
#maybe divide all of the AAC/GAAC columns by the protein length 

In [89]:
df.shape

(16099, 16)

In [90]:
df = df.replace([np.inf, -np.inf], np.nan)

In [91]:
nan_counts = df.isna().sum()
print(nan_counts)
nan_counts.unique()

local_gap_compressed_percent_id      0
scaled_local_query_percent_id        0
scaled_local_symmetric_percent_id    0
query_align_len                      0
query_align_cov                      0
subject_align_len                    0
subject_align_cov                    0
bit_score                            0
m_protein_len                        0
t_protein_len                        0
protein_match                        0
GAAC_alphatic                        0
GAAC_aromatic                        3
GAAC_postivecharge                   0
GAAC_negativecharge                  1
GAAC_uncharge                        0
dtype: int64


array([0, 3, 1])

In [107]:
df.columns[36:]

Index(['QSOrder_Grantham.Xd.2', 'QSOrder_Grantham.Xd.3'], dtype='object')

In [110]:
log_cols = df.columns[36:]
df[log_cols] = df[log_cols].apply(lambda x: np.log(x))

In [92]:
df = df.dropna(axis=1, how='any')

In [93]:
nan_counts = df.isna().sum()
nan_counts

local_gap_compressed_percent_id      0
scaled_local_query_percent_id        0
scaled_local_symmetric_percent_id    0
query_align_len                      0
query_align_cov                      0
subject_align_len                    0
subject_align_cov                    0
bit_score                            0
m_protein_len                        0
t_protein_len                        0
protein_match                        0
GAAC_alphatic                        0
GAAC_postivecharge                   0
GAAC_uncharge                        0
dtype: int64

In [24]:
df.shape

(17108, 17)

Use MRMR to select for the best features. Going to start by grouping into different categories of features generated from iFeature Omega.

In [123]:
#use MRMR to select for the best features from PseKRAAC
df_subset = df.loc[:, df.columns != 'protein_match']
print(type(df_subset))

# select top 10 features using mRMR
from mrmr import mrmr_classif
selected_features = mrmr_classif(X=df_subset.iloc[:,10:], y=df['protein_match'], K=20)

selected_features

<class 'pandas.core.frame.DataFrame'>


100%|██████████| 20/20 [00:07<00:00,  2.83it/s]


['type3A_T1.G.1_T1.G.1_gap2',
 'CTDC_solventaccess.G1',
 'Geary_BEGF750102.lag2',
 'type7_T1.G.2_T1.G.2_gap2',
 'type9_T1.G.1_T1.G.1_gap2',
 'CTDC_secondarystruct.G1',
 'type2_T1.G.1_T1.G.1_gap2',
 'APAAC_Pc1.G',
 'CTDD_normwaalsvolume.3.residue50',
 'CTDT_secondarystruct.Tr1221',
 'APAAC_Pc1.V',
 'QSOrder_Schneider.Xd.3',
 'CTDC_hydrophobicity_ARGP820101.G2',
 'CTDD_hydrophobicity_ENGD860101.3.residue75',
 'CTDD_charge.2.residue0',
 'CTDC_hydrophobicity_ENGD860101.G2',
 'CTDD_secondarystruct.1.residue100',
 'CTDD_secondarystruct.1.residue0',
 'APAAC_Pc2.Hydrophobicity.1',
 'GDPC_alphaticr.alphaticr_x']

In [124]:
best_features_df = df[[feature for feature in selected_features]]

#concatenates original feature vector back into the dataframe
df = pd.concat([df.iloc[:, :11], best_features_df], axis=1)
df

,local_gap_compressed_percent_id,scaled_local_query_percent_id,scaled_local_symmetric_percent_id,query_align_len,query_align_cov,subject_align_len,subject_align_cov,bit_score,m_protein_len,t_protein_len,...,APAAC_Pc1.V,QSOrder_Schneider.Xd.3,CTDC_hydrophobicity_ARGP820101.G2,CTDD_hydrophobicity_ENGD860101.3.residue75,CTDD_charge.2.residue0,CTDC_hydrophobicity_ENGD860101.G2,CTDD_secondarystruct.1.residue100,CTDD_secondarystruct.1.residue0,APAAC_Pc2.Hydrophobicity.1,GDPC_alphaticr.alphaticr_x
0,0.333333,0.267380,0.254453,148,0.791444,163,0.791262,127,206,187,...,1.458032,1.095793,0.974189,0.895744,0.907767,0.960391,1.000000,0.907767,-0.418672,1.017965
1,0.268482,0.190083,0.189300,248,0.683196,271,0.740437,132,366,363,...,1.036049,1.078083,1.111504,1.147146,0.991803,1.248937,1.002762,0.991803,0.721630,1.405023
2,0.333333,0.295082,0.291498,218,0.893443,221,0.884000,158,250,244,...,0.767388,0.992939,0.847094,0.986844,0.976000,0.903319,1.012448,0.976000,27.063868,0.948021
3,0.275281,0.182156,0.152174,185,0.687732,197,0.525333,48,375,269,...,1.320507,1.027001,1.152081,0.958679,0.717333,1.120330,1.003731,0.717333,-0.547390,1.328926
4,0.331915,0.245283,0.242613,239,0.751572,238,0.732308,202,325,318,...,1.753038,1.008479,1.048352,0.990743,0.978462,0.881497,1.000000,0.978462,0.579979,1.188936
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17103,0.331707,0.294372,0.282158,202,0.874459,218,0.868526,232,251,231,...,0.897562,1.073475,0.965212,0.970336,0.920319,1.057680,1.008734,0.920319,-0.310949,1.070204
17104,0.304878,0.188679,0.163666,166,0.626415,170,0.491329,125,346,265,...,1.287880,1.090877,1.117224,0.944200,0.765896,0.968633,0.988439,0.765896,0.340536,0.869565
17105,0.344828,0.169972,0.192308,180,0.509915,200,0.738007,53,271,353,...,1.112860,1.001577,1.018956,1.170603,1.302583,1.089045,0.995440,1.302583,-3.313974,0.835708
17106,0.352174,0.239645,0.242152,230,0.680473,227,0.685801,279,331,338,...,0.872070,1.030476,0.857453,1.008222,1.021148,0.976361,1.015015,1.021148,-2.922809,0.940892


In [94]:
#choosing 80/20 split instead of 85/15 because of volume of data

dev, test = sklearn.model_selection.train_test_split(df, test_size=0.15, random_state=1)

train, val = sklearn.model_selection.train_test_split(dev, test_size=0.15, random_state=1)

print(dev.shape)
print(test.shape)
print(train.shape)
print(val.shape)

(13684, 14)
(2415, 14)
(11631, 14)
(2053, 14)


In [95]:
#ID target and features, separate into separate arrays

target = 'protein_match'
input_features = [columns for columns in df]
input_features.remove(target)

In [96]:
#split X and y

dev_X = dev[input_features].values
test_X = test[input_features].values

dev_y = dev[target].values.reshape(-1,1)
test_y = test[target].values.reshape(-1,1)  

print(dev_X.shape, test_X.shape, dev_y.shape, test_y.shape)

(13684, 13) (2415, 13) (13684, 1) (2415, 1)


In [97]:
#same thing for training and validation data

train_X = train[input_features].values
val_X = val[input_features].values

train_y = train[target].values.reshape(-1,1)
val_y = val[target].values.reshape(-1,1) 

Scale the data

In [98]:
scaler = sklearn.preprocessing.StandardScaler()
dev_X = scaler.fit_transform(dev_X)
test_X = scaler.fit_transform(test_X)
train_X = scaler.fit_transform(train_X)
val_X = scaler.fit_transform(val_X)

Train the model

In [99]:
#Random Forest

#hyperparameters determiend with optuna
model = sklearn.ensemble.RandomForestClassifier(n_estimators=150, max_depth=None, max_samples=0.5,
                                                max_features=0.5, min_weight_fraction_leaf=0.000215,
                                               min_samples_split=10)

model.fit(train_X, train_y.ravel())

RandomForestClassifier(max_features=0.5, max_samples=0.5, min_samples_split=10,
                       min_weight_fraction_leaf=0.000215, n_estimators=150)

Test the model, report relevant statistics

In [100]:
score = model.score(val_X, val_y)
print('Mean accuracy score is: {}'.format(score))

preds = model.predict(test_X)
print(preds)

Mean accuracy score is: 0.6244520214320507
['No' 'Yes' 'No' ... 'Yes' 'Yes' 'No']


In [101]:
from sklearn.metrics import f1_score, confusion_matrix, classification_report

In [102]:
f1_score(preds, test_y, pos_label='Yes')

0.6308724832214765

In [ ]:
%matplotlib notebook

In [ ]:
#confusion matrix

confusion_matrix = sklearn.metrics.confusion_matrix(preds, test_y)
sklearn.metrics.ConfusionMatrixDisplay(confusion_matrix).plot()